# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [116]:
import pandas as pd
import os
import numpy as np

In [117]:
#DATA_FOLDER = "/home/vinz/Desktop/ADA/ADA2017-Tutorials/02 - Intro to Pandas/Data" # Use the data folder provided in Tutorial 02 - Intro to Pandas.
DATA_FOLDER = "./Data"

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average* per year of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [118]:
# Setup for location of the dataset of the task 2
MICROBIOME_FOLDER = DATA_FOLDER + "/microbiome"

In [119]:
# Write your answer here
# TODO: Should the data from the spreadsheets be merged in different columns instead of adding indexes as we are doing now
#        Should at least the tissue and stool from the same group of patient be merged?


### Basic analysis of the file formatting:

***
**For the files MIDn.xls with n in [1,9]**

**column 1:**

We see that the first column in the files contain the scientific classification of the microbiomes
Altough it could be kept as a single string, it would have more meaning if splitted
The scientific classification contains the following subdivisions (https://en.wikipedia.org/wiki/Taxonomic_rank): 

       Domain, Kingdom, Phylum, Class, Order, Family, Genus, Species
       
 With this classification, we have a problem: only 6 strings are given in the data set, whilst we have 8 potential divisions in the classification.
 
 After analysis, it seems that Kingdom is not used with bacterias (https://en.wikipedia.org/wiki/Bacteria)
 and always the same as the Phylum in Archea(https://en.wikipedia.org/wiki/Archaea). Therefore, **we will not use Kingdom as a division**.
 
 Also, Species only has a sense in the Eucaryote Domain, for which we have no data, so **we will not use Species as a division**.
 
 In addition, after playing with the data, we found cases where the Family can be named: "Incertae Sedis", which would be classified as the family and genus in our classification. Therefore, we need to check for those cases and re-concatenate the strings to return a proper list in every cases.

In [120]:
# Lets keep those names in a list for further use
scientific_classification = ["domain", "phylum", "class", "order", "family", "genus"]

**Column 2:**

We see that the second column contains an integer value. We can suppose that this value is the number of samples
containing the genus defined by the first column. There's no title to the column, so we don't know yet where
those values came from.

***

**For the file "Metadata.xls"**

**Column 1:**

Titled **"BARCODE"**, give xls file identifier for which the two other columns give more information

**Column 2:**

Titled **"GROUP"**, gives the groups from which each dataset has been sampled. Two informations are contained in those groups, the first is given by "NEC", "Control" or "EXTRACTION CONTROL" and the second is the numbering of the group (which is likely a phase of test), either "1" or "2". We will split those informations in two column, because while using the dataset, we might want to combine all the "NEC" patients or all the patient for a specific phase.

**Column 3:**

Titled **"SAMPLE"**, gives the type of sample that was taken, either tissue, stool or NA. Each group had both types of samples taken. 

***

### Desired formating of the data after analysis

The simple analysis above tells us what are the columns that we will want in our Data Frame

1. **6** column for classification, 1 for each classifier in the "scientific_classification" list. This will caracterise each microbiome individually.

2. **1** column will contain the value associated with each microbiome measurement.

3. **2** columns describing the group of the sample from the metadata. The first column will be called **Group Type** and will contain the "NEC", "Control" or "EXTRACTION CONTROL" value. The second column will be called **Group Phase** and will contain either "1", "2" or "unknow", "unknow" will be used for the "EXCTRACTON CONTROL" group.

4. **1** column describing the type of sample taken from the metadata. This column will be called **Sample** This will be either "tissue", "stool" or "unknow", "unknow" in the case of the EXTRACTION CONTROL group.

For a total of **10** columns


In [121]:
metadata_list = ["group_type", "group_phase", "sample"]

In [122]:
df_col_list = scientific_classification + metadata_list + ["value"]

In [123]:
print(len(df_col_list))
print(df_col_list)

10
['domain', 'phylum', 'class', 'order', 'family', 'genus', 'group_type', 'group_phase', 'sample', 'value']


***
### We want to extract the metadata from the metadata.xls file

The metadata is needed before we start extracting the data from the MB files to create the DataFrame with all the desired columns.


In [124]:
#The actual name of the excel sheet is "Sheet1" and not "Sheet 1" as in the other files.
metadata_raw = pd.read_excel(MICROBIOME_FOLDER+"/metadata.xls", sheetname='Sheet1', header=0)
metadata_raw.columns = metadata_raw.columns.str.lower()


In [125]:
## Extract group phase and group type
group = metadata_raw["group"]
group_type = []
group_phase = []
for index in range(len(group)):                # Index progression must be upward!
    # A special case for the extraction control, we don't want to split it
    if group[index] == "EXTRACTION CONTROL":
        group_type.append(group[index])
        group_phase.append("unknow")
    else:
        splitted_group = group[index].split()
        group_type.append(splitted_group[0])
        group_phase.append(splitted_group[1])
        

In [126]:
metadata_raw['group_type'] = pd.Series(group_type)
metadata_raw['group_phase'] = pd.Series(group_phase)
metadata = metadata_raw.drop("group",  axis=1)
metadata

,barcode,sample,group_type,group_phase
0,MID1,NaN,EXTRACTION CONTROL,unknow
1,MID2,tissue,NEC,1
2,MID3,tissue,Control,1
3,MID4,tissue,NEC,2
4,MID5,tissue,Control,2
5,MID6,stool,NEC,1
6,MID7,stool,Control,1
7,MID8,stool,NEC,2
8,MID9,stool,Control,2


In [127]:
## make sure SAMPLE doesn't contain NaN
metadata["sample"].fillna(value="unknow", inplace=True)
metadata

,barcode,sample,group_type,group_phase
0,MID1,unknow,EXTRACTION CONTROL,unknow
1,MID2,tissue,NEC,1
2,MID3,tissue,Control,1
3,MID4,tissue,NEC,2
4,MID5,tissue,Control,2
5,MID6,stool,NEC,1
6,MID7,stool,Control,1
7,MID8,stool,NEC,2
8,MID9,stool,Control,2


The format of the metadata is now as we want it

***
### We now want to extract the data from the datasheets

** We create a function to extract the classifications **

We want to extract the classifications from the first column of the datasheets. We will need to do string parsing. Regex are a great tool to remove the unwanted characters from the format. The only unwanted characters are the double quotes ". Also, as said before, we need to manage the case where we have the name "Incertae Sedis" as a Family (4th position)

In [128]:
import re

# TODO: this could be made more general, for every potential cases where there's more that 6 elements
# Next function replace " in the classifier_string, split the string to have an indexable list and return it
def get_classifiers(classifier_string):
    classified_list = re.subn("\"", "", classifier_string)[0].split()
    
    # Special case management
    if len(classified_list) > 6:
        # We join the first extra location with the family
        classified_list[4] = classified_list[4] + "_" + classified_list[5]
        del classified_list[5]
        # Use a loop to allow for harbitrary number of words in the genus and for safety
        while len(classified_list) > 6:
            classified_list[5] += "_" + classified_list[6]
            del classified_list[6]
        
    return classified_list
    # Check if the 4th value is "Incertae"
#    if "Incertae" in classified_list:
#        classified_list[4] = classified_list[4] + "_" + classified_list[5]
#        if classified_list[7]:
#            classified_list[5] = classified_list[6] + "_" + classified_list[7]
#            classified_list.pop()
#            classified_list.pop()
#        else:
#            classified_list[5] = classified_list[6]
#            classified_list.pop()
#    return classified_list

** We can now attempt to create the desired dataframe **

In [129]:
# In this section, we test the analysis of a single datasheet

test_data = pd.read_excel(MICROBIOME_FOLDER+"/MID1.xls", sheetname='Sheet 1', header=None)
test_data.columns = ["raw_classification", "value"]
classifier_df = pd.DataFrame(columns=df_col_list)
classifier_array = []
for row_data in test_data.itertuples():
    classifier_array.append(get_classifiers(row_data.raw_classification))
classifier_series = pd.DataFrame(classifier_array, columns=scientific_classification)
classifier_series["value"] = test_data["value"]
classifier_series

,domain,phylum,class,order,family,genus,value
0,Archaea,Crenarchaeota,Thermoprotei,Desulfurococcales,Desulfurococcaceae,Ignisphaera,7
1,Archaea,Crenarchaeota,Thermoprotei,Desulfurococcales,Pyrodictiaceae,Pyrolobus,2
2,Archaea,Crenarchaeota,Thermoprotei,Sulfolobales,Sulfolobaceae,Stygiolobus,3
3,Archaea,Crenarchaeota,Thermoprotei,Thermoproteales,Thermofilaceae,Thermofilum,3
4,Archaea,Euryarchaeota,Methanomicrobia,Methanocellales,Methanocellaceae,Methanocella,7
5,Archaea,Euryarchaeota,Methanomicrobia,Methanosarcinales,Methanosarcinaceae,Methanimicrococcus,1
6,Archaea,Euryarchaeota,Methanomicrobia,Methanosarcinales,Methermicoccaceae,Methermicoccus,1
7,Archaea,Euryarchaeota,Archaeoglobi,Archaeoglobales,Archaeoglobaceae,Ferroglobus,1
8,Archaea,Euryarchaeota,Archaeoglobi,Archaeoglobales,Archaeoglobaceae,Geoglobus,1
9,Archaea,Euryarchaeota,Halobacteria,Halobacteriales,Halobacteriaceae,Haloplanus,1


In [138]:
# Loop over all the datasheets
clean_data = pd.DataFrame(columns=df_col_list)
for metadata_row in metadata.itertuples():
    raw_data = pd.read_excel(MICROBIOME_FOLDER+"/"+metadata_row.barcode+".xls", sheetname='Sheet 1', header=None)
    # Change column names to something clearer
    raw_data.columns = ["raw_classification", "value"]
    
    # For each datasheet create a local classified set of data
    classifier_array = []
    for row_data in raw_data.itertuples():
        classifier_array.append(get_classifiers(row_data.raw_classification))
    local_classified = pd.DataFrame(classifier_array, columns=scientific_classification)
    # To the local set of data add the columns that are not the classification
    local_classified["value"]       = test_data["value"]
    local_classified["group_type"]  = metadata_row.group_type
    local_classified["group_phase"] = metadata_row.group_phase
    local_classified["sample"]      = metadata_row.sample
    # Append this local set of data to "clean_data", which is the merge of the data from each measurements
    clean_data = clean_data.append(local_classified, ignore_index=True)

clean_data

,class,domain,family,genus,group_phase,group_type,order,phylum,sample,value
0,Thermoprotei,Archaea,Desulfurococcaceae,Ignisphaera,unknow,EXTRACTION CONTROL,Desulfurococcales,Crenarchaeota,unknow,7
1,Thermoprotei,Archaea,Pyrodictiaceae,Pyrolobus,unknow,EXTRACTION CONTROL,Desulfurococcales,Crenarchaeota,unknow,2
2,Thermoprotei,Archaea,Sulfolobaceae,Stygiolobus,unknow,EXTRACTION CONTROL,Sulfolobales,Crenarchaeota,unknow,3
3,Thermoprotei,Archaea,Thermofilaceae,Thermofilum,unknow,EXTRACTION CONTROL,Thermoproteales,Crenarchaeota,unknow,3
4,Methanomicrobia,Archaea,Methanocellaceae,Methanocella,unknow,EXTRACTION CONTROL,Methanocellales,Euryarchaeota,unknow,7
5,Methanomicrobia,Archaea,Methanosarcinaceae,Methanimicrococcus,unknow,EXTRACTION CONTROL,Methanosarcinales,Euryarchaeota,unknow,1
6,Methanomicrobia,Archaea,Methermicoccaceae,Methermicoccus,unknow,EXTRACTION CONTROL,Methanosarcinales,Euryarchaeota,unknow,1
7,Archaeoglobi,Archaea,Archaeoglobaceae,Ferroglobus,unknow,EXTRACTION CONTROL,Archaeoglobales,Euryarchaeota,unknow,1
8,Archaeoglobi,Archaea,Archaeoglobaceae,Geoglobus,unknow,EXTRACTION CONTROL,Archaeoglobales,Euryarchaeota,unknow,1
9,Halobacteria,Archaea,Halobacteriaceae,Haloplanus,unknow,EXTRACTION CONTROL,Halobacteriales,Euryarchaeota,unknow,1


** Now lets replace all the None values by unknow **

In [131]:
for item in clean_data.iteritems():
    item[1].fillna(value="unknow", inplace=True)

In [132]:
clean_data.head(10)

,class,domain,family,genus,group_phase,group_type,order,phylum,sample,value
0,Thermoprotei,Archaea,Desulfurococcaceae,Ignisphaera,unknow,EXTRACTION CONTROL,Desulfurococcales,Crenarchaeota,unknow,7
1,Thermoprotei,Archaea,Pyrodictiaceae,Pyrolobus,unknow,EXTRACTION CONTROL,Desulfurococcales,Crenarchaeota,unknow,2
2,Thermoprotei,Archaea,Sulfolobaceae,Stygiolobus,unknow,EXTRACTION CONTROL,Sulfolobales,Crenarchaeota,unknow,3
3,Thermoprotei,Archaea,Thermofilaceae,Thermofilum,unknow,EXTRACTION CONTROL,Thermoproteales,Crenarchaeota,unknow,3
4,Methanomicrobia,Archaea,Methanocellaceae,Methanocella,unknow,EXTRACTION CONTROL,Methanocellales,Euryarchaeota,unknow,7
5,Methanomicrobia,Archaea,Methanosarcinaceae,Methanimicrococcus,unknow,EXTRACTION CONTROL,Methanosarcinales,Euryarchaeota,unknow,1
6,Methanomicrobia,Archaea,Methermicoccaceae,Methermicoccus,unknow,EXTRACTION CONTROL,Methanosarcinales,Euryarchaeota,unknow,1
7,Archaeoglobi,Archaea,Archaeoglobaceae,Ferroglobus,unknow,EXTRACTION CONTROL,Archaeoglobales,Euryarchaeota,unknow,1
8,Archaeoglobi,Archaea,Archaeoglobaceae,Geoglobus,unknow,EXTRACTION CONTROL,Archaeoglobales,Euryarchaeota,unknow,1
9,Halobacteria,Archaea,Halobacteriaceae,Haloplanus,unknow,EXTRACTION CONTROL,Halobacteriales,Euryarchaeota,unknow,1


** Now lets manage the index of the DataFrame **

For now, we consider that every column but the value can be considered as a metadata. 

In [155]:
indexed_data = clean_data.set_index(list(scientific_classification + ["group_type", "group_phase", "sample"]))
indexed_data

value
domain   phylum                class                 order                    family                           genus              group_type         group_phase sample      
Archaea  Crenarchaeota         Thermoprotei          Desulfurococcales        Desulfurococcaceae               Ignisphaera        EXTRACTION CONTROL unknow      unknow     7
                                                                              Pyrodictiaceae                   Pyrolobus          EXTRACTION CONTROL unknow      unknow     2
                                                     Sulfolobales             Sulfolobaceae                    Stygiolobus        EXTRACTION CONTROL unknow      unknow     3
                                                     Thermoproteales          Thermofilaceae                   Thermofilum        EXTRACTION CONTROL unknow      unknow     3
         Euryarchaeota         Methanomicrobia       Methanocellales          Methanocellaceae                 Methanocella       EXTRACTION CONTROL unknow      unknow     7
                                                     Methanosarcinales        Methanosarcinaceae               Methanimicrococcus EXTRACTION CONTROL unknow      unknow     1
                                                                              Methermicoccaceae                Methermicoccus     EXTRACTION CONTROL unknow      unknow     1
                               Archaeoglobi          Archaeoglobales          Archaeoglobaceae                 Ferroglobus        EXTRACTION CONTROL unknow      unknow     1
                                                                                                               Geoglobus          EXTRACTION CONTROL unknow      unknow     1
                               Halobacteria          Halobacteriales          Halobacteriaceae                 Haloplanus         EXTRACTION CONTROL unknow      unknow     1
                                                                                                               Haloquadratum      EXTRACTION CONTROL unknow      unknow     4
                                                                                                               Halosimplex        EXTRACTION CONTROL unknow      unknow     1
                                                                                                               Natronobacterium   EXTRACTION CONTROL unknow      unknow     2
                                                                                                               Natronomonas       EXTRACTION CONTROL unknow      unknow     4
                               Methanococci          Methanococcales          Methanocaldococcaceae            Methanotorris      EXTRACTION CONTROL unknow      unknow     1
                               Methanopyri           Methanopyrales           Methanopyraceae                  Methanopyrus       EXTRACTION CONTROL unknow      unknow    12
                               Thermoplasmata        Thermoplasmatales        Thermoplasmatales_incertae_sedis Thermogymnomonas   EXTRACTION CONTROL unknow      unknow     2
Bacteria Actinobacteria        Actinobacteria        Actinomycetales          Acidothermaceae                  Acidothermus       EXTRACTION CONTROL unknow      unknow     2
                                                                              Beutenbergiaceae                 Salana             EXTRACTION CONTROL unknow      unknow     1
                                                                              Bogoriellaceae                   Bogoriella         EXTRACTION CONTROL unknow      unknow     1
                                                                              Cellulomonadaceae                Tropheryma         EXTRACTION CONTROL unknow      unknow     1
                                                                              Corynebacteriaceae               Turicella          EXTRACTION CONTROL unknow      unkno

In [134]:
indexed_data.index

MultiIndex(levels=[['Control', 'EXTRACTION CONTROL', 'NEC'], ['1', '2', 'unknow'], ['stool', 'tissue', 'unknow'], ['Archaea', 'Bacteria'], ['Acidobacteria', 'Actinobacteria', 'Aquificae', 'Bacteroidetes', 'Caldiserica', 'Chlamydiae', 'Chlorobi', 'Chloroflexi', 'Chrysiogenetes', 'Crenarchaeota', 'Cyanobacteria', 'Deferribacteres', 'Deinococcus-Thermus', 'Euryarchaeota', 'Firmicutes', 'Fusobacteria', 'Gemmatimonadetes', 'Lentisphaerae', 'Nitrospira', 'OD1', 'OP11', 'Planctomycetes', 'Proteobacteria', 'Spirochaetes', 'Synergistetes', 'TM7', 'Tenericutes', 'Thermodesulfobacteria', 'Thermotogae', 'Verrucomicrobia'], ['Acidobacteria_Gp16', 'Actinobacteria', 'Alphaproteobacteria', 'Anaerolineae', 'Aquificae', 'Archaeoglobi', 'Bacilli', 'Bacteroidetes_incertae_sedis', 'Bacteroidia', 'Betaproteobacteria', 'Caldilineae', 'Caldisericia', 'Chlamydiae', 'Chlorobia', 'Chloroflexi', 'Chrysiogenetes', 'Clostridia', 'Cyanobacteria', 'Deferribacteres', 'Dehalococcoidetes', 'Deinococci', 'Deltaproteobact

In [135]:
indexed_data.index.is_unique

True

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [136]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

Name,Labels,Units,Levels,Storage,NAs
pclass,,,3,integer,0
survived,Survived,,,double,0
name,Name,,,character,0
sex,,,2,integer,0
age,Age,Year,,double,263
sibsp,Number of Siblings/Spouses Aboard,,,double,0
parch,Number of Parents/Children Aboard,,,double,0
ticket,Ticket Number,,,character,0
fare,Passenger Fare,British Pound (\243),,double,1
cabin,,,187,integer,0


For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [137]:
# Write your answer here